### Combining GDP data with the jobs report

Brian Dew

Updated: September 20, 2020

----

Notes:

The basic idea here is to use the BEA population estimates plus CPS employment rate and hours worked trends to estimate GDP per hour of work. BLS does this process for its productivity and costs report, using much more comprehensive data, however this approximation has proven decent over time.

The hard part to this measure is getting hours worked right. All of the published measures are either too-low-frequency or don't have a broad enough definition of workers. I want to capture all workers, regardless of full-time or part-time status or of whether they work for the private sector. Also want to capture second and third jobs. As a result, I selected total actual hours worked from the CPS microdata, specifically finding the trend using x13as with default settings. Hours worked from the CPS microdata have issues around holidays falling in the reference week, and also do not capture hours worked for some important categories of labor such as self-employed persons.

In [1]:
%matplotlib inline
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

qtrs = {1: 1, 2: 1, 3: 1, 4: 2, 5:2, 6:2, 7:3, 8:3, 9:3, 10:4, 11:4, 12:4}

In [2]:
avghrs = lambda x: np.average(x.HRSACTT.replace(-1, 0), weights=x.WGT)
aah, epop = {}, {}
cols = ['HRSACTT', 'LFS', 'MONTH']
for year in range(1989, 2023):
    wgt = 'PWSSWGT' if year >= 1998 else 'BASICWGT'
    df = pd.read_feather(cps_dir / f'cps{year}.ft', 
                         columns=cols + [wgt]).rename({wgt: 'WGT'}, axis=1)
    emp = df.query('LFS == "Employed"')
    ah = (emp.groupby('MONTH').apply(avghrs))
    aah.update({pd.to_datetime(f'{year}-{month}-01'): value 
                for month, value in list(zip(ah.index, ah.values))})
    ep = (emp.groupby('MONTH').WGT.sum() / df.groupby('MONTH').WGT.sum())
    epop.update({pd.to_datetime(f'{year}-{month}-01'): value 
                 for month, value in list(zip(ep.index, ep.values))})

In [3]:
sm = x13_arima_analysis(pd.Series(epop))
epop_sa = sm.seasadj.resample('QS').mean()
sm = x13_arima_analysis(pd.Series(aah))
aah_sa = sm.trend.resample('QS').mean()

/home/brian/miniconda3/lib/python3.8/site-packages/statsmodels/tsa/x13.py:203: X13Warning: NOTE: Unable to test AO2020.Mar due to regression matrix singularity.
       The effect of this outlier is already accounted for by other regressors 
       (usually user-specified or previously identified outliers).

 NOTE: Unable to test LS2020.May due to regression matrix singularity.
       The effect of this outlier is already accounted for by other regressors 
       (usually user-specified or previously identified outliers).

 NOTE: Unable to test LS2020.Mar due to regression matrix singularity.
       The effect of this outlier is already accounted for by other regressors 
       (usually user-specified or previously identified outliers).

 NOTE: Unable to test AO2020.Apr due to regression matrix singularity.
       The effect of this outlier is already accounted for by other regressors 
       (usually user-specified or previously identified outliers).

 NOTE: Unable to test LS2020.May d

In [4]:
gdp_code = ('T10106', ['A191RX'])
pop_code = ('T70100', ['B230RC'])
cd = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC'].iloc[-1]
rgdp = nipa_df(retrieve_table('T10106')['Data'], ['A191RX'])
gdp = rgdp / rgdp.iloc[-1] * cd
pop = nipa_df(retrieve_table(pop_code[0])['Data'], pop_code[1]).sort_index()

df = pd.DataFrame()
df['epop'] = epop_sa * 100
df['pop'] = pop['B230RC'] / 1000
#df['hours'] = fred_df('PRS84006023')
df['hours'] = aah_sa
df['gdp'] = gdp['A191RX']
df['input'] = (df['pop'] * (df['epop'] / 100)) * (df['hours'] * 52)
df['gdpinp'] = df['gdp'] / df['input']
datetxt = dtxt(df['gdpinp'].dropna().index[-1])['qtr1']
write_txt(text_dir / 'gdpjobs_ltdt.txt', datetxt)
df.to_csv(data_dir / 'gdpjobslvl.csv', index_label='date')

df['epop_c'] = ((df['pop'] * df['epop'].mean()) * df['hours']) * df['gdpinp']
df['pop_c'] = ((df['pop'].mean() * df['epop']) * df['hours']) * df['gdpinp']
df['hours_c'] = ((df['pop'] * df['epop']) * df['hours'].mean()) * df['gdpinp']

df_g = growth_rate(df.dropna())

df_g['pop_contr'] = df_g['gdp'] - df_g['pop_c']
df_g['epop_contr'] = df_g['gdp'] - df_g['epop_c']
df_g['hours_contr'] = df_g['gdp'] - df_g['hours_c']
df_g['prod'] = (df_g['gdp'] - df_g['pop_contr'] - 
                df_g['epop_contr'] - df_g['hours_contr'])
result = df_g[['pop_contr', 'epop_contr', 'hours_contr', 'prod']].round(2)
result.to_csv(data_dir / 'gdpjobs.csv', index_label='date')

In [5]:
data = df.dropna()

ltval = f"\${data['gdpinp'].iloc[-1]:.2f}"
prval = f"\${data['gdpinp'].iloc[-2]:.2f}"
prdate = dtxt(data['gdpinp'].index[-2])['qtr1']
prdt = '2019-10-01'
prval2 = f"\${data.loc[prdt, 'gdpinp']:.2f}"
prval3 = f"\${data.loc['2015-10-01', 'gdpinp']:.2f}"
val89 = f"\${data.loc['1989-01-01', 'gdpinp']:.2f}"
gdpval = f"\${data['gdp'].iloc[-1] / 1_000:,.0f}"
gdppr = f"\${data.loc[prdt, 'gdp'] / 1_000:,.0f}"
agghrs = f"{data['input'].iloc[-1] / 1_000:,.0f}"
hrspr = f"{data.loc[prdt, 'input'] / 1_000:,.0f}"

text = (f'In {datetxt}, real GDP was equivalent to roughly {ltval} '+
        f'per hour of work, compared to {prval} in {prdate}, {prval2} '+
        f'in 2019 Q4, {prval3} in 2015 Q4, and {val89} in the first quarter '+
        'of 1989.\n\nComparing the latest data to the pre-COVID data covering '+
        f'2019 Q4, annualized real GDP is {gdpval} billion in the latest data '+
        f'and {gdppr} billion in 2019 Q4. Aggregate hours worked total '+
        f'{agghrs} billion in the latest quarter and {hrspr} billion '+
        'in 2019 Q4.')
write_txt(text_dir / 'gdp_per_hour.txt', text)
print(text, '\n\n')

poplt = value_text(result['pop_contr'].iloc[-1], style='contribution_to', 
                   ptype='pp', digits=2)
poppr = value_text(result.loc[prdt, 'pop_contr'], style='contribution', 
                   ptype='pp', digits=2, casual=True)
emplt = value_text(result['epop_contr'].iloc[-1], style='contribution', 
                   ptype='pp', digits=2)
emppr = value_text(result.loc[prdt, 'epop_contr'], style='contribution', 
                   ptype='pp', digits=2, casual=True)
hrslt = value_text(result['hours_contr'].iloc[-1], style='contribution_to', 
                   ptype='pp', digits=2, casual=True)
hrspr = value_text(result.loc[prdt, 'hours_contr'], style='contribution', 
                   ptype='pp', digits=2, casual=True)
prodlt = value_text(result['prod'].iloc[-1], style='contribution_to', 
                    ptype='pp', digits=2)
prodpr = value_text(result.loc[prdt, 'prod'], style='contribution_of', 
                    ptype='pp', digits=2, casual=True)

text = (f'In {datetxt}, population growth {poplt} annualized GDP growth, '+
        f'and, for comparison, {poppr} in 2019 Q4. Changes in the '+
        f'employed share of the population {emplt} in the latest quarter, '+
        f'and {emppr} in the fourth quarter of 2019. Changes in average '+
        f'hours worked {hrslt} GDP growth in the latest quarter and {hrspr} '+
        f'in 2019 Q4. Lastly, productivity {prodlt} GDP growth in {datetxt}, '+
        f'compared to {prodpr} in 2019 Q4.')
write_txt(text_dir / 'gdpjobsch.txt', text)
print(text)

In 2022 Q3, real GDP was equivalent to roughly \$82.01 per hour of work, compared to \$81.75 in 2022 Q2, \$77.26 in 2019 Q4, \$74.99 in 2015 Q4, and \$50.24 in the first quarter of 1989.

Comparing the latest data to the pre-COVID data covering 2019 Q4, annualized real GDP is \$25,663 billion in the latest data and \$24,630 billion in 2019 Q4. Aggregate hours worked total 313 billion in the latest quarter and 319 billion in 2019 Q4. 


In 2022 Q3, population growth contributed 0.37 percentage point to annualized GDP growth, and, for comparison, added 0.59 percentage point in 2019 Q4. Changes in the employed share of the population contributed 0.84 percentage point in the latest quarter, and added 1.61 percentage points in the fourth quarter of 2019. Changes in average hours worked added 0.09 percentage point to GDP growth in the latest quarter and added 0.35 percentage point in 2019 Q4. Lastly, productivity contributed 1.26 percentage points to GDP growth in 2022 Q3, compared to a redu

In [6]:
df

,epop,pop,hours,gdp,input,gdpinp,epop_c,pop_c,hours_c
1989-01-01,48.433198,246.460,37.432260,1.167350e+07,232348.021891,50.241459,2.220927e+07,2.689238e+07,2.243204e+07
1989-04-01,48.388751,247.017,37.458875,1.176260e+07,232824.848740,50.521254,2.239934e+07,2.703653e+07,2.258719e+07
1989-07-01,48.467951,247.698,37.481756,1.184975e+07,233991.685211,50.641756,2.252842e+07,2.716196e+07,2.274064e+07
1989-10-01,48.425322,248.374,37.484984,1.187310e+07,234444.109030,50.643626,2.259268e+07,2.714141e+07,2.278349e+07
1990-01-01,48.348861,248.936,37.479494,1.200285e+07,234569.216302,51.169752,2.287569e+07,2.737606e+07,2.303584e+07
...,...,...,...,...,...,...,...,...,...
2021-10-01,47.808361,332.584,37.130975,2.564337e+07,307004.611997,83.527639,4.942508e+07,4.377722e+07,4.967665e+07
2022-01-01,48.279745,332.749,37.115571,2.553815e+07,310056.770838,82.366031,4.874169e+07,4.357596e+07,4.949334e+07
2022-04-01,48.537375,332.940,37.121424,2.550121e+07,311939.398783,81.750523,4.841285e+07,4.348798e+07,4.941396e+07
2022-07-01,48.637137,333.241,37.129911,2.566329e+07,312934.673027,82.008455,4.862062e+07,4.372485e+07,4.971666e+07
